In [26]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("adult_with_headers.csv")
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
df.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64

In [4]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [5]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Apply Standard Scaling to numerical features
scaler_standard = StandardScaler()
df[['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']] = scaler_standard.fit_transform(df[['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']])

# Apply Min-Max Scaling to numerical features
scaler_minmax = MinMaxScaler()
df[['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']] = scaler_minmax.fit_transform(df[['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']])


In [11]:
# Apply One-Hot Encoding to categorical variables with less than 5 categories
df = pd.get_dummies(df, columns=['marital_status', 'relationship', 'race', 'sex'], drop_first=True)


In [13]:
from sklearn.preprocessing import LabelEncoder

# Use Label Encoding for 'education' column
label_encoder = LabelEncoder()
df['education'] = label_encoder.fit_transform(df['education'])


In [15]:
# Create new feature 'capital_change' representing net capital gain/loss
df['capital_change'] = df['capital_gain'] - df['capital_loss']

# Create new feature 'work_hours_per_education' representing average work hours per education level
df['work_hours_per_education'] = df['hours_per_week'] / df['education_num']


In [17]:
# Apply log transformation to 'fnlwgt' feature
import numpy as np

df['fnlwgt'] = np.log1p(df['fnlwgt'])


In [19]:
from sklearn.ensemble import IsolationForest

# Identify outliers using Isolation Forest
isolation_forest = IsolationForest(contamination=0.05)
outlier_preds = isolation_forest.fit_predict(df[['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']])

# Remove outliers
data_clean = df[outlier_preds == 1]


In [20]:
data_clean

,age,workclass,fnlwgt,education,education_num,occupation,capital_gain,capital_loss,hours_per_week,native_country,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,capital_change,work_hours_per_education
0,0.301370,State-gov,0.043349,9,0.800000,Adm-clerical,0.021740,0.0,0.397959,United-States,...,0,0,0,0,0,0,1,1,0.021740,0.497449
1,0.452055,Self-emp-not-inc,0.047110,9,0.800000,Exec-managerial,0.000000,0.0,0.122449,United-States,...,0,0,0,0,0,0,1,1,0.000000,0.153061
2,0.287671,Private,0.129372,11,0.533333,Handlers-cleaners,0.000000,0.0,0.397959,United-States,...,0,0,0,0,0,0,1,1,0.000000,0.746173
3,0.493151,Private,0.140690,1,0.400000,Handlers-cleaners,0.000000,0.0,0.397959,United-States,...,0,0,0,0,1,0,0,1,0.000000,0.994898
4,0.150685,Private,0.200070,9,0.800000,Prof-specialty,0.000000,0.0,0.397959,Cuba,...,0,0,1,0,1,0,0,0,0.000000,0.497449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.136986,Private,0.153926,7,0.733333,Tech-support,0.000000,0.0,0.377551,United-States,...,0,0,1,0,0,0,1,0,0.000000,0.514842
32557,0.315068,Private,0.092124,11,0.533333,Machine-op-inspct,0.000000,0.0,0.397959,United-States,...,0,0,0,0,0,0,1,1,0.000000,0.746173
32558,0.561644,Private,0.090596,11,0.533333,Adm-clerical,0.000000,0.0,0.397959,United-States,...,0,1,0,0,0,0,1,0,0.000000,0.746173
32559,0.068493,Private,0.120889,11,0.533333,Adm-clerical,0.000000,0.0,0.193878,United-States,...,1,0,0,0,0,0,1,1,0.000000,0.363520


In [27]:
import ppscore as pps

# Calculate PPS matrix
pps_matrix = pps.matrix(data_clean)

# Display PPS matrix
print(pps_matrix)


                            x                         y  ppscore  \
0                         age                       age  1.00000   
1                         age                 workclass  0.00825   
2                         age                    fnlwgt  0.00000   
3                         age                 education  0.00000   
4                         age             education_num  0.00000   
..                        ...                       ...      ...   
836  work_hours_per_education               race_ Other      NaN   
837  work_hours_per_education               race_ White      NaN   
838  work_hours_per_education                 sex_ Male      NaN   
839  work_hours_per_education            capital_change      NaN   
840  work_hours_per_education  work_hours_per_education  1.00000   

               case  is_valid_score               metric  baseline_score  \
0    predict_itself            True                 None        0.000000   
1    classification            